<h1> Pred Morph using LSTM

In [1]:
from baseline_LSTM_MORPH import *
import copy

In [2]:
# Load data
sentences, morphs = load_data("UD_French-Sequoia/fr_sequoia-ud-train.conllu")

In [3]:
# Build vocabulary
word_counts = Counter(word for sentence in sentences for word in sentence)
word_to_ix = {word: i + 1 for i, word in enumerate(word_counts)}  # +1 for padding
word_to_ix['<PAD>'] = 0

morph_counts = set(trait for trait_list in morphs for trait in trait_list if trait != '_')
morph_to_ix = {trait: i+1 for i, trait in enumerate(morph_counts)}
morph_to_ix['_'] = 0

In [4]:
# Model parameters
embedding_dim = 64
hidden_dim = 128
morphs_size = len(morph_to_ix)
epochs = 10
batch_size = 16

In [5]:
# Create dataset and data loader
dataset = MORPHDataset(sentences, morphs, word_to_ix, morph_to_ix)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [6]:
model = MORPH_Model(len(word_to_ix), embedding_dim, hidden_dim, len(morph_to_ix))
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

<h2> Training model

In [7]:
#Entrainement du modèle
for epoch in range(epochs):
    total_loss = 0
    for batch in data_loader:
        sentence_batch, morphs_batch = batch
        model.zero_grad()
        morph_scores = model(sentence_batch)
        loss = loss_function(morph_scores.view(-1, len(morph_to_ix)), morphs_batch.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

Epoch 1, Loss: 1.683001868213926
Epoch 2, Loss: 1.220599662406104
Epoch 3, Loss: 1.0896537946803229
Epoch 4, Loss: 0.9844149155276162
Epoch 5, Loss: 0.9252076351216861
Epoch 6, Loss: 0.887850849543299
Epoch 7, Loss: 0.8319853356906346
Epoch 8, Loss: 0.8000874155334063
Epoch 9, Loss: 0.7751765581113952
Epoch 10, Loss: 0.7325477204152516


In [8]:
loss, accuracy = evaluate_model(model, data_loader, loss_function, morph_to_ix)
print(f"Train Accuracy : {accuracy:.4f}")
print(f"Train loss : {loss:.4f}")

Train Accuracy : 0.8424
Train loss : 0.7206


<h2> Validation

In [9]:
sentences_dev, morphs_dev = load_data("UD_French-Sequoia/fr_sequoia-ud-dev.conllu")

In [10]:
dataset_dev = MORPHDataset(sentences_dev, morphs_dev, word_to_ix, morph_to_ix)
data_loader_dev = DataLoader(dataset_dev, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [11]:
loss, accuracy = evaluate_model(model, data_loader_dev, loss_function, morph_to_ix)
print(f"Validation Accuracy : {accuracy:.4f}")
print(f"Validation loss : {loss:.4f}")

Validation Accuracy : 0.8281
Validation loss : 0.8735


<h3> Testing model

In [12]:
sentences_test, morphs_test = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu")

In [13]:
dataset_test = MORPHDataset(sentences_test, morphs_test, word_to_ix, morph_to_ix)
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [14]:
loss, accuracy = evaluate_model(model, data_loader_test, loss_function, morph_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")
print(f"Test loss : {loss:.4f}")

Test Accuracy : 0.8453
Test loss : 0.7650
